# Rayo: Your daily news help

In [24]:
import requests
import re
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
from bs4 import BeautifulSoup
from transformers import AutoModelForCausalLM, AutoTokenizer, BartConfig, #TFBartForConditionalGeneration

In [2]:
#A better architecture will be created to select news category later. For now we stick to tech news
default_url = "https://www.bbc.com"
url = "https://www.bbc.com/news/technology"
'''
category = input("Enter a category:")
if category == "technology":
    url = "https://www.bbc.com/news/technology"
elif category == "business":
    url = "https://www.bbc.com/news/business"
elif category == "entertainment":
    url = "https://www.bbc.com/news/entertainment_and_arts"
elif category == "health":
    url = "https://www.bbc.com/news/health"
elif category == 'science':
    url = "https://www.bbc.com/news/science_and_environment"
elif category == 'sports':
    url = "https://www.bbc.com/sport"
elif category == 'world':
    url = "https://www.bbc.com/news/world"

    '''
# Set a time threshold for "recent" news (e.g., up to a day)
one_day_ago = datetime.now() - timedelta(days=1)

# Send an HTTP request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the page using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the news articles on the page
    articles = soup.find_all('a',class_='qa-heading-link lx-stream-post__header-link')

    # Initialize lists to store article headlines and links
    news_headlines = []
    news_links = []

    # Loop through the articles and extract relevant information
    for article in articles:
        link = article.find('href')
        " ".join(article.text.split())
        news_headlines.append(article.text)

        #Extract link
        string_soup = str(article).replace('<html><body>', '').replace('</body></html>', '').replace('<p>', '').replace('</p>', '')
        pattern = r'href="([^"]*)"'
        match = re.search(pattern, string_soup)
        if match:
            extracted_url = match.group(1)
            news_links.append(default_url + extracted_url)
        else:
            print("No match found.")
else:
    print("Failed to retrieve the webpage. Status code:", response.status_code)


In [3]:
content = []

for url in news_links:
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        text = ""
        for paragraph in soup.find_all('p'):
            text += paragraph.get_text() + " "
        content.append(text)
    else:
        print("Failed to retrieve the webpage. Status code:", response.status_code)

In [4]:
daily_news_data = pd.DataFrame({'Headline': news_headlines, 'Link': news_links, 'Content': content})

In [8]:
daily_news_data['Content'][0]

'A Polish tech entrepreneur\'s global project, aimed at getting more children into computer programming, has been endorsed by Pope Francis.  Miron Mironiuk, founder of artificial intelligence company Cosmose AI, is drawing on his own experience of coding transforming his life. He said the "Code with Pope" initiative would bridge "the glaring disparities in education" across the globe.  It is hoped the Pope\'s involvement will attract Catholic countries. "We believe that the involvement of the Pope will help to convince them to spend some time and use this opportunity to learn programming for free," Mr Mironiuk told the BBC.  The initiative will champion access to coding education through a free online learning platform for students aged 11-15 across Europe, Africa and Latin America.  After 60 hours of dedicated learning, children will be equipped with the basics of Python, one of the world\'s most popular coding languages.  In the digital age, programming skills have become as fundamen

In [19]:
from transformers import BartForConditionalGeneration, BartTokenizer

In [23]:
# Load the pretrained model and tokenizer
config = BartConfig.from_pretrained('facebook/bart-large-cnn')
model = TFBartForConditionalGeneration(config=config)
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")


#model_name = "facebook/bart-large-cnn"
#tokenizer = BartTokenizer.from_pretrained(model_name)
#model = BartForConditionalGeneration.from_pretrained(model_name)


# Input text to summarize
input_text = daily_news_data['Content'][0]

# Tokenize and summarize
#input_ids = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True, padding="max_length")
#summary_ids = model.generate(input_ids, max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)

#summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
#print(summary)

NameError: name 'TFBartForConditionalGeneration' is not defined

In [ ]:
#from transformers import TFBartForConditionalGeneration, BartConfig,AutoTokenizer

#config = BartConfig.from_pretrained('facebook/bart-large-cnn')
#model = TFBartForConditionalGeneration(config=config)
#tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
#summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)
#print (summarizer)

In [ ]:

# Input text to summarize
input_text = daily_news_data['Content'][0]

# Tokenize and summarize
input_ids = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True, padding="max_length")
summary_ids = model.generate(input_ids, max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)

summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary)


KeyboardInterrupt: 

In [ ]:
# Create a summarization pipeline using the "facebook/bart-large-cnn" model
#pipe = pipeline("summarization", model="facebook/bart-large-cnn")
#print (pipe)

All model checkpoint layers were used when initializing TFBartForConditionalGeneration.

All the layers of TFBartForConditionalGeneration were initialized from the model checkpoint at facebook/bart-large-cnn.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


In [ ]:
#from transformers import TFBartForConditionalGeneration, BartConfig,AutoTokenizer

#config = BartConfig.from_pretrained('facebook/bart-large-cnn')
#model = TFBartForConditionalGeneration(config=config)
#tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
#summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)
#print (summarizer)

In [ ]:
#summary = pipe(daily_news_data['Content'][0], max_length=150, min_length=50, do_sample=False)

In [ ]:
#summary[0]['summary_text']

'A new £6.1m residential building equipped with the latest technology is due to open at a college for students with disabilities. The accommodation at National Star College will act as a "smart house" and includes voice-activated technology. It will allow students to adapt artificial intelligence (AI) to suit their personal needs.'

In [ ]:
#def generate_summary(content):
 #   summary = pipe(content, max_length=150, min_length=50, do_sample=False)
  #  summarized_text = summary[0]['summary_text']
   # return summarized_text

In [ ]:
#summarized_text = generate_summary(daily_news_data['Content'][0])
#print (summarized_text)